# Running MD Simulations Using Amber

## Background

Amber is one of the original MD simulation packages.

This means good things:
 - It has been optimised, de-bugged and had its functionality honed over decades.
 - It's very fast, especially on GPUs.

But also some less good things:
 - No GUI, a command line interface used via a terminal.
 - Reads and writes files in formats that are not immediately easy to understand.

A web search will lead you to many good Amber tutorials on the Internet, here we are going to just give a quick introduction.

## The Amber MD engines

Amber has two "core" MD engines (programs that actually run MD simulations):
 - **sander**: the original, huge range of functionality, does not run on GPUs
 - **pmemd**: A bit more modern, does all the most common types of simulation but not everything **sander** can, but is very fast, especially on GPUs ("**pmemd.cuda**")

## Running simulations: input and output files

First a nore about a peculiarity of **Amber** - it is very relaxed about the names of files. While many programs need a file containing data of a particular type to have a particular *extension* (think *.docx, *.pdf, etc.), **Amber** does not require this. However, over the years, certain conventions have arisen, mainly to help users remember what sort of information is in what file. **Also: using "approved" extensions can also be vital if you need your file to be readable by other programs.**

To run a simulation, **pmemd** requires information from three input files:

1. A file containing information about the current coordinates (and maybe velocities) of each atom. Typically this file is given the extension *".inpcrd"*, or *".rst7"* if it is designed to be human-readable, or *".ncrst"* if it has the newer binary (netCDF) format.
2. A file containing all the other data that define the model of the molecular system, in particular the *topology* (what is connected to what) and the *parameters* (equilibrium bond lengths, bond force constants, partial charges on each atom, etc.). Commonly this file is given the extension *".prmtop"*.
3. A file containing the instructions on how the simulation should be run. Typically this file is given the extension *".in"*.

*Note: for certain types of simulation, additional input files may be required.*

The simulation will produce at least two output files:

1. A file containing information about the final coordinates (and maybe velocities) of each atom. Since this file may be used as the coordinates input file for a further simulation, it's commonly called the *restart file*, and has the extension *".rst7"* (human-readable format) or *".ncrst"* (binary format).
2. A log file with (lots) of information about how the simulation has progressed. Typically this is given the extension *".out"* or *".log"*.

Several other output files may also be generated, depending on the type of simulation and on options specified in the input instructions file. 

In particular, an MD simulation frequently is instructed to produce a *trajectory file*, which records the coordinates of the atoms at regularly-spaced timepoints throughout the simulation - in effect, a form of movie file. If this data is saved in the efficient binary netCDF format (recommended), then the filename is typically given the extension *".nc"*.

## The MD Workflow: Relaxation and Equilibration

No matter how careful you have been in the process of preparing a molecular system for simulation, almost inevitably the initially-built conformation will contain areas with poor geometry, clashes between atoms, etc. - in other words, parts of the system will have large forces being exerted between atoms, and so a high potential energy. If you just start an MD simulation from this situation, the regions of high potential energy will become regions of high kinetic energy - i.e., the atoms will quickly have high velocities. The core algorithms that model the dynamics of the system are only mathematically stable up to certain velocities - beyond this they can "blow up". 

To avoid this, a careful relaxation and equilibration process is required. This typically involves two types of simulation: *energy minimisation*, and *restrained molecular dynamics*. 

In *energy minimisation*, small steps (alterations in atom coordinates) are made that are analytically calculated to reduce the magnitude of the forces present in the system. Two algorithms are typically used: *steepest descent* works best when a few very bad forces dominate, while *conjugate gradient* is more efficient when the forces are more similar in magnitude. It's common therefore to start with some steepest descent optimisation, then switch to conjugate gradient later. The optimisation process is iterative, and is generally set to stop when either no forces over a certain threshold remain, or some maximum number of steps has been reached.

In *restrained molecular dynamics* the atoms (or a chosen subset of them) are "tethered" to set positions (typically, the positions they occupy at the start of the simulation), by additional harmonic forces added to the system. This means that they can move away from their initial positions if they really need to, but are unlikely to get far. This helps to avoid the "blowing up" phenomenon. Very often a series of restrained MD simulations are run one after the other. In each one the forces restraining the atoms are loosened further, and/or the number of atoms they are being applied to is reduced, until finally it's safe to remove them entirely.

It's important to note that there is no guaranteed recipie for succesful relaxation and equilibration of a molecular system, typically each molecular simulation research group has its favorite protocol.

## Running a basic energy minimisation job.

In the folder for this workshop you will find copies of the coordinates and topology files for Mcl-1 (5fdr_A) that you prepared in the previous workshop. In addition you will find the file `min.in`, which contains the instructions to run a simple energy minimisation job.

1. Take a look at `min.in` (either by using "cat" or "less" in a terminal window, or double-clicking on the file in the Jupyter Lab browser window). The top half of the file is hopefully fairly readable, but unfortunately is only there for the user's benefit - **pmemd** (which is the program we will use) does not read this; it's only interested ij the lines after the one that starts "&cntrl", which you will see is much more cryptic, you may need to refer to the [Amber manual](https://ambermd.org/doc12/Amber24.pdf) to decode it.

2. Run the energy minimization job directly from the command line (i.e., in your terminal window). Type:

        pmemd -i min.in -c 5fdr_A.inpcrd -p 5fdr_A.prmtop -o min.log -r 5fdr_A_min.ncrst

The job will take maybe 30 seconds to run. When it's complete, you will find the new restart file (`5fdr_A_min.rst7`) and log file (`min.log`) have appeared in your directory. 

3. The restart file is in a binary format you can't read, but the log file is just a text file, so take a look. Basically it is divided into three parts:

   1. An introduction that gives details of every parameter that controlled how the job ran: what the input script was, what the file names were, what the parameters that the user set were, and (very importantly) the default values given to many other parameters that the user did not specify. Itvis divided into three sections.
   2. A section (beginning **"4. RESULTS"**) that reports on the progress of the simulation, reporting metrics like the values of energy components and which atom was experiencing the largest force, at each step the user asked to be logged.
   3. A final section that summarises the simulation, including some performance data.

If things have gone OK, if you scan through the middle section you should see that the total energy of the system, and of energy components, is reducing each step. Lines like:

       NSTEP       ENERGY          RMS            GMAX         NAME    NUMBER
        400      -7.9128E+04     9.7282E-01     4.0335E+01     HB        546

mean "At step 400, the total energy of the system was -7.9*10^4 kilojoules per mole. The Root-mean-square gradient of the forces in the system was 0.972 kilocalories per mole per Angstrom. The greatest force component had a magnitude of 40.3 kilocalories per mole per Angstrom, and was being experienced by atom number 546, which has the name HB".
   
When things go wrong (as they may do, if the starting structure has some very bad geometry), this information about which atom is experiencing the greatest force can be very helpful for debugging the issue. However you need to be able to work out what atom 546 actually is in the molecular structure. This is where the PDB format file `5fdr_A_amber.pdb` comes in useful - e.g. if you scan through this you will se that atom number 546 is the HB atom of THR35:

        ATOM    546  HB  THR    35      48.358  32.962  44.082  1.00  0.00           H 


Towards the end of the file you will see:

          Maximum number of minimization cycles reached.

                            FINAL RESULTS

           NSTEP       ENERGY          RMS            GMAX         NAME    NUMBER
           1000      -8.1473E+04     3.4380E-01     1.5753E+01     HA        544

The energy minimisation has terminated because the set maximum of 1000 cycles have been run. Note that a "perfect" energy minimisation would finish with no net force acting on any atom in the system, but in reality this is never achieved, and one just looks for a reasonable value for the RMS force gradient. As a rule of thumb, a value below 0.5 kcal/mol/Ang. is probably good enough if the plan is to move on to MD, so this particular minimization has reached this target.

## Running a basic MD simulation

1. Take a look at the file `md.in` which provides the neccessary instructions for a simple MD simulation. As for the minimisation script, only the data after `&cntrl` is actually read by **pmemd**, the top half is just to explain the key points to a reader, and hopefully is sufficiently explanatory.
2. Run the MD simulation directly from the command line in your terminal window. The simulation starts from the final coordinates of the system from the preceeding energy minimisation step:
   ```
   
       pmemd -i md.in -c 5fdr_A_min.ncrst -p 5fdr_A.prmtop -o md.log \
          -r 5fdr_A_md.ncrst -x 5fdr_A_md.nc

       
    ```

   The *"-x"* argument is the name of the *trajectory* file that will be generated. Depending on the computational power of the platform you are working on, this MD job may take 1-5 minutes to complete.

3. When it completes, take a look at the log file, `md.log`. The general format is the same as for the minimisation log file: an introductory section, a section recording key parameters at regular time intervals, and then a summary section. The data in the middle section should look largely familiar, only as now this is an MD simulation there is time and temperature to report. In addition, the simulation has been instructed to attempt to maintain a constant pressure, which it does by contracting or expanding the size of the periodic box, resulting in changes in the system density. If you scan through the file you will see the system temperature rising to 310 Kelvin and the density of the system increasing to about 1 g/cm^3, where they settle, plus or minus some fluctuation.

## A better relaxation/equilibration workflow

As it happens, this very simple relaxation/equilibration workflow seems to have succeeded: the MD simulation has reached the target temperature and a realistic density without "blowing up". But for security. in most cases it's better to adopt a more cautious strategy. Here you will try out a protocol adapted from one used by the Wheeler group at the University of Montana, not because it is superior to any other, but because it happens to demonstrate a range of MD simulation options and controls, and has been implemented as a linux *bash script* that can be automatically generated for any molecular system.

1. The script can be generated from information in your initially-generated coordinate and parameter/topology files, using the `mesmy` utility:
```
mesmy -i 5fdr_A.inpcrd -p 5fdr_A.prmtop > relax.sh

```

2. Take a look at `relax.sh`. Depending on your familiarity with *bash* scripts this may be more or less easy to understand, but in essence the first part of the script generates a set of nine *pmemd* input (instructions) files, tailored for the details of the specific system, and then the second half runs the jobs, one after the other. There is also some logic in the script so that if the first run of the job fails for some reason, a restart can pick up from where the last one ended, and successful steps are not needlessly re-run.

